In [1]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
Root = "/content/drive/MyDrive/RAVDESS DATA"
os.chdir(Root)

In [3]:
ls

speech-emotion-recognition-ravdess-data/


In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [7]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result           

In [8]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [9]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/RAVDESS DATA/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [16]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [18]:
x_train

array([[-6.54364929e+02,  5.59474030e+01,  1.00320959e+01, ...,
         5.19384639e-06,  4.13871658e-06,  2.09481800e-06],
       [-4.06886902e+02,  3.92465687e+00, -3.57620888e+01, ...,
         2.10690731e-03,  1.39552972e-03,  5.71330020e-04],
       [-4.74077240e+02,  1.22089348e+01, -2.70769711e+01, ...,
         5.35305240e-04,  3.11566429e-04,  2.22294897e-04],
       ...,
       [-6.02701782e+02,  6.38962364e+01, -6.86303377e-01, ...,
         6.55650147e-05,  2.93123740e-05,  1.56385413e-05],
       [-5.09563599e+02,  2.71493607e+01, -1.39932795e+01, ...,
         1.02189624e-04,  6.04939014e-05,  2.96137096e-05],
       [-6.34414917e+02,  5.94407043e+01, -6.70986843e+00, ...,
         7.27277948e-05,  3.44498148e-05,  1.66608024e-05]])

In [19]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [20]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [21]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [22]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [23]:
#Predict for the test set
y_pred=model.predict(x_test)

In [24]:
y_pred

array(['disgust', 'disgust', 'calm', 'calm', 'disgust', 'disgust',
       'disgust', 'happy', 'fearful', 'fearful', 'calm', 'calm', 'calm',
       'happy', 'happy', 'disgust', 'disgust', 'disgust', 'calm',
       'disgust', 'calm', 'happy', 'fearful', 'disgust', 'disgust',
       'disgust', 'happy', 'fearful', 'calm', 'calm', 'calm', 'calm',
       'disgust', 'happy', 'disgust', 'fearful', 'happy', 'fearful',
       'disgust', 'calm', 'happy', 'calm', 'happy', 'fearful', 'disgust',
       'fearful', 'happy', 'fearful', 'disgust', 'disgust', 'calm',
       'calm', 'calm', 'happy', 'calm', 'fearful', 'calm', 'happy',
       'disgust', 'happy', 'happy', 'fearful', 'calm', 'calm', 'disgust',
       'fearful', 'fearful', 'disgust', 'fearful', 'disgust', 'calm',
       'happy', 'happy', 'disgust', 'fearful', 'disgust', 'happy',
       'disgust', 'disgust', 'disgust', 'disgust', 'happy', 'fearful',
       'happy', 'disgust', 'fearful', 'fearful', 'happy', 'calm', 'happy',
       'happy', 'dis

In [25]:
from sklearn.metrics import accuracy_score, f1_score

In [26]:
f1_score(y_test, y_pred,average=None)

array([0.79120879, 0.64285714, 0.725     , 0.65346535])

In [27]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,disgust
1,disgust,disgust
2,calm,calm
3,calm,calm
4,disgust,disgust
5,disgust,disgust
6,fearful,disgust
7,disgust,happy
8,fearful,fearful
9,fearful,fearful


In [28]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [29]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/RAVDESS DATA/speech-emotion-recognition-ravdess-data/Actor_01/03-01-03-01-02-02-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['disgust'], dtype='<U7')

In [30]:
feature

array([[-6.35018066e+02,  4.89507713e+01,  2.72784328e+00,
         1.72580223e+01, -2.41734004e+00, -6.83206134e-03,
        -7.16416597e+00, -1.37409506e+01, -8.42472553e+00,
         1.62511170e+00, -5.93697739e+00,  2.89293098e+00,
        -6.72187948e+00,  2.43873429e+00, -4.60308409e+00,
        -6.11418009e+00, -1.10687983e+00, -2.53769445e+00,
        -5.91165733e+00, -2.70875597e+00, -2.44323778e+00,
        -6.02066946e+00, -6.14662826e-01, -3.78392696e+00,
        -3.91626811e+00, -1.30044806e+00, -3.33906841e+00,
        -1.28022206e+00, -4.05895567e+00, -1.89656532e+00,
        -4.55924606e+00, -2.86183119e+00, -2.62580824e+00,
        -1.64863431e+00, -4.60142422e+00, -3.05698037e+00,
        -3.14702010e+00, -2.58076882e+00, -2.19267058e+00,
        -3.05325127e+00,  6.38672531e-01,  6.77346826e-01,
         7.06679046e-01,  6.84544444e-01,  6.45216763e-01,
         6.99994147e-01,  6.97099447e-01,  7.17782080e-01,
         7.43450820e-01,  7.28027880e-01,  7.47324228e-0

In [31]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 69.79%
